In [1]:
 # Dependencies
import requests
import json
from config import api_key_open
from pprint import pprint
import pandas as pd
from pathlib import Path

In [2]:
# Read the csv file and create a dataframe
file_path = Path('Downloads/Resources/organised_world_data.csv')
countries_df = pd.read_csv(file_path)
countries_df.head()

,Country,Capital/Major City,Density(P/Km2),Population,Urban_population,Land Area(Km2),Agricultural Land( %),Forested Area (%),Co2-Emissions,GDP,Life expectancy,Birth Rate,Maternal mortality ratio,Infant mortality,Latitude,Longitude
0,Afghanistan,Kabul,60,38041754,9797273,652230,58.1,2.1,8672,1.910135e+10,64.5,32.49,638,47.9,33.939110,67.709953
1,Albania,Tirana,105,2854191,1747593,28748,43.1,28.1,4536,1.527808e+10,78.5,11.78,15,7.8,41.153332,20.168331
2,Algeria,Algiers,18,43053054,31510100,2381741,17.4,0.8,150006,1.700000e+11,76.7,24.28,112,20.1,28.033886,1.659626
3,Angola,Luanda,26,31825295,21061025,1246700,47.5,46.3,34693,9.463542e+10,60.8,40.73,241,51.6,-11.202692,17.873887
4,Argentina,Buenos Aires,17,44938712,41339571,2780400,54.3,9.8,201348,4.500000e+11,76.5,17.02,39,8.8,-38.416097,-63.616672


In [68]:
# Add the columns to a new dataframe
country_aq_df=pd.DataFrame(countries_df["Country"])
country_aq_df["City"]=countries_df["Capital/Major City"]
country_aq_df["CO"]=""
country_aq_df["NH3"]=""
country_aq_df["NO"]=""
country_aq_df["NO2"]=""
country_aq_df["O3"]=""
country_aq_df["PM10"]=""
country_aq_df["PM2_5"]=""
country_aq_df["SO2"]=""
country_aq_df["AQI"]=""
country_aq_df.head()

,Country,City,CO,NH3,NO,NO2,O3,PM10,PM2_5,SO2,AQI
0,Afghanistan,Kabul,,,,,,,,,
1,Albania,Tirana,,,,,,,,,
2,Algeria,Algiers,,,,,,,,,
3,Angola,Luanda,,,,,,,,,
4,Argentina,Buenos Aires,,,,,,,,,


In [14]:
# Set the base URL to get latitude and longitude of cities
base_url = f"http://api.openweathermap.org/data/2.5/weather?appid={api_key_open}&q="

In [15]:
# Loop over the new dataframe, get latitude and longitude for each city and then use new url to get the air pollutants data
for index, row in country_aq_df.iterrows():
    try:
        city_url=base_url+row["City"]+", "+row["Country"]
        response=requests.get(city_url).json()
        lat=response["coord"]["lat"]
        lon=response["coord"]["lon"]
        air_quality_url=f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={api_key_open}"
        response_data=requests.get(air_quality_url).json()
        country_aq_df.loc[index,"CO"]=response_data["list"][0]["components"]["co"]
        country_aq_df.loc[index,"NH3"]=response_data["list"][0]["components"]["nh3"]
        country_aq_df.loc[index,"NO"]=response_data["list"][0]["components"]["no"]
        country_aq_df.loc[index,"NO2"]=response_data["list"][0]["components"]["no2"]
        country_aq_df.loc[index,"O3"]=response_data["list"][0]["components"]["o3"]
        country_aq_df.loc[index,"PM10"]=response_data["list"][0]["components"]["pm10"]
        country_aq_df.loc[index,"PM2_5"]=response_data["list"][0]["components"]["pm2_5"]
        country_aq_df.loc[index,"SO2"]=response_data["list"][0]["components"]["so2"]
        country_aq_df.loc[index,"AQI"]=response_data["list"][0]["main"]["aqi"]
    except:
        print(row["Country"])                                                                                   

In [16]:
# Print the final dataframe
country_aq_df.head(10)

,Country,City,CO,NH3,NO,NO2,O3,PM10,PM2_5,SO2,AQI
0,Afghanistan,Kabul,186.92,0.47,0.08,0.41,120.16,8.24,3.64,0.24,3
1,Albania,Tirana,260.35,7.03,0.71,8.91,231.74,23.26,21.09,0.85,5
2,Algeria,Algiers,317.1,0.08,1.84,16.11,158.79,206.1,39.97,3.16,5
3,Angola,Luanda,2643.59,34.96,3.69,56.21,234.6,351.28,267.81,34.33,5
4,Argentina,Buenos Aires,327.11,1.3,1.02,28.45,15.74,3.73,2.11,12.99,1
5,Armenia,Yerevan,178.58,1.12,0.12,0.67,108.72,1.41,1.32,0.18,3
6,Australia,Canberra,208.62,0.31,0.02,4.07,32.9,2.71,2.16,0.35,1
7,Austria,Vienna,236.99,0.55,0.14,2.29,56.51,2.65,2.32,0.62,1
8,Azerbaijan,Baku,210.29,0,0.3,4.16,115.87,7.56,6.4,2.3,3
9,The Bahamas,"Nassau, Bahamas",230.31,0.43,0.03,0.87,73.67,1.55,1.12,0.44,2


In [17]:
# Export the new dataframe to csv
country_aq_df.to_csv('Downloads/Resources/country_air_quality.csv',index=False,header=True)